## Setup

All libs that will be use in this note

In [1]:
import numpy as np
from glob import glob
import random

## Preprocessing

There are two folders that contains the inputs of the problem **Traveling Salesman Problem** they are different so let's split it.

### Test Data

In [2]:
PATH_TEST = 'instancias/instancias_teste/'
DATA_TEST = glob(PATH_TEST+'*.txt')
print(DATA_TEST)

['instancias/instancias_teste/ch150.txt', 'instancias/instancias_teste/bays29.txt', 'instancias/instancias_teste/berlin52.txt', 'instancias/instancias_teste/ch130.txt', 'instancias/instancias_teste/brazil58.txt', 'instancias/instancias_teste/swiss42.txt', 'instancias/instancias_teste/bayg29.txt', 'instancias/instancias_teste/bier127.txt', 'instancias/instancias_teste/descricao.txt']


### TSP Cup Data

In [3]:
PATH_CUP = 'instancias/instancias_tsp_cup/'
DATA_CUP = glob(PATH_CUP+'*.txt')
print(DATA_CUP)

['instancias/instancias_tsp_cup/tsp1.txt', 'instancias/instancias_tsp_cup/tsp2.txt', 'instancias/instancias_tsp_cup/tsp3.txt', 'instancias/instancias_tsp_cup/descricao.txt']


## Math

Some mathematics functions that will be useful 

In [4]:
def euclidian_distance(p1, p2):
    return int(np.linalg.norm(p1-p2).round())

In [5]:
print(euclidian_distance(np.array([1,1]), np.array([1,10])))

9


## Graph

We'll use a graph as data structure in order to representation our problem, where each city is a vertex and a connection is representation by a edge. 

In [6]:
class Graph:
    def __init__(self, num_vertices):
        self._adjacency_matrix = np.zeros((num_vertices, num_vertices), dtype=np.int64)
        
    def build(self, file, _type='tsp_test'):
        if _type == 'tsp_cup':
            self._handle_tsp_cup(file)
        elif _type == 'tsp_test':
            self._handle_tsp_test(file)
        else:
            print('Undefined type')
        
    def _handle_tsp_cup(self, file):
        data = []
        with open(file, 'r') as f:
            for line in f:
                try:
                    _, x, y = list(filter(lambda x: x!='', line.split(' ')))
                    data.append(np.array((float(x), float(y[:-1]))))
                except:
                    pass
        for i, p1 in enumerate(data):
            for j, p2 in enumerate(data):
                self._adjacency_matrix[i][j] = euclidian_distance(p1, p2)
    
    def _handle_tsp_test(self, file):
        data = []
        with open(file, 'r') as f:
            _input = f.readlines()
            _input = [_.strip() for _ in _input]
            _input = [_.split() for _ in _input]
            for i, array in enumerate(_input[3:]):
                try:
                    _ = [int(_) for _ in array]
                    if len(_) > 0:
                        data.append(_)
                except:
                    pass
        self._adjacency_matrix = data
    
    def num_vertices(self):
        return len(self._adjacency_matrix)
    
    def get_cost(self, _list_path_nodes):
        path = [self._adjacency_matrix[current][_next] for current, _next in zip(_list_path_nodes[1:], _list_path_nodes[:-1])]
        return sum(path)

## Optimizations Solutions

Some functions that will give us solutions for our problem.

## Helpers

Below functions that will be userful to generation solutions

In [7]:
def pick_element(_list, nodes):
    for index, x in enumerate(_list):
        if not nodes[index]:
            return index, x
    return None

In [8]:
def get_min_distance(_distances, nodes):
    min_index, min_val = pick_element(_distances, nodes)
    for index, x in enumerate(_distances):
        if min_val > x:
            min_val = x if not nodes[index] else min_val
            min_index = index if not nodes[index] else min_index
    return min_val, min_index

In [60]:
def get_cost_swap(graph, cost, _list, i, k):
    cost_swap = cost
    if i == k:
        return cost_swap
    # make sure that k is always higher than i
    if i > k:
        i, k = k, i
    if i+1 == k:
        prev, _next = i-1 if i > 0 else i, k+1 if k < graph.num_vertices() - 1 else k
        cost_swap -= graph.get_cost([_list[prev], _list[i], _list[k], _list[_next]])
        
        prev, _next = i-1 if i > 0 else k, k+1 if k < graph.num_vertices() - 1 else i
        cost_swap += graph.get_cost([_list[prev], _list[k], _list[i], _list[_next]])
    else:
        prev_i, _next_i = i-1 if i > 0 else i, i+1
        prev_k, _next_k = k-1, k+1 if k < graph.num_vertices() - 1 else k
        
        cost_swap -= graph.get_cost([_list[prev_i], _list[i], _list[_next_i]])
        cost_swap -= graph.get_cost([_list[prev_k], _list[k], _list[_next_k]])
        
        prev_i, _next_i = i-1 if i > 0 else k, i+1
        prev_k, _next_k = k-1, k+1 if k < graph.num_vertices() - 1 else i
        
        cost_swap += graph.get_cost([_list[prev_i], _list[k], _list[_next_i]])
        cost_swap += graph.get_cost([_list[prev_k], _list[i], _list[_next_k]])
    return cost_swap

### Nearest Neighbor

In [63]:
def nearest_neighbor(graph, initial):
    solution = []
    current_node = initial
    nodes = [False for x in range(graph.num_vertices())]
    nodes[current_node] = True
    flag = True
    solution.append(current_node)
    while flag:
        distances = graph._adjacency_matrix[current_node]
        min_distance, min_index = get_min_distance(distances, nodes)
        current_node = min_index
        nodes[current_node] = True
        solution.append(current_node)
        if all(nodes):
            solution.append(initial)
            flag = False
    return graph.get_cost(solution), solution

### Best Insertion

In [111]:
def best_insertion(graph, initial):
    current_cost, current_solution = initial
    index = np.random.randint(graph.num_vertices())
    for i in range(graph.num_vertices()):
        cost = get_cost_swap(graph, current_cost, current_solution, i, index)
        if current_cost > cost:
            current_solution[i], current_solution[index] = current_solution[index], current_solution[i]
            current_cost = cost
    return graph.get_cost(current_solution), current_solution

### Swap

In [112]:
def first_swap(graph, initial):
    current_cost, current_solution = initial
    flag = True
    while flag:
        flag = False
        for i in range(graph.num_vertices()):
            for k in range(i+1, graph.num_vertices()):
                cost = get_cost_swap(graph, current_cost, current_solution, i, k) 
                if current_cost > cost:
                    current_solution[i], current_solution[k] = current_solution[k], current_solution[i]
                    current_cost = cost
                    return graph.get_cost(current_solution), current_solution
    return graph.get_cost(current_solution), current_solution

### Variable Neighborhood Descent

In [154]:
def vnd(graph, initial, itermax):
#     initial = np.arange(graph.num_vertices())
#     np.random.shuffle(initial)
    index_random = np.random.randint(graph.num_vertices())
    _, initial = nearest_neighbor(graph, index_random)
    initial = np.array(initial[:-1])

    _iter = 0
    current_cost, current_solution = graph.get_cost(initial), initial.tolist()

    while _iter < itermax:
        is_better = True
        while is_better:
            is_better = False
            cost, solution = best_insertion(graph, (current_cost, current_solution))
            if current_cost > cost:
                current_cost, current_solution = cost, solution
                is_better = True
                _iter = 0
            else:
                cost, solution = first_swap(graph, (current_cost, current_solution))
                if current_cost > cost:
                    current_cost, current_solution = cost, solution
                    is_better = True
                    _iter = 0
        _iter += 1
    current_solution = current_solution + [current_solution[0]]
    return graph.get_cost(current_solution), current_solution

## Test

For each test we'll build a graph with the adjacency matrix representing our cities.

### TSP TEST

In [155]:
print(DATA_TEST)

['instancias/instancias_teste/ch150.txt', 'instancias/instancias_teste/bays29.txt', 'instancias/instancias_teste/berlin52.txt', 'instancias/instancias_teste/ch130.txt', 'instancias/instancias_teste/brazil58.txt', 'instancias/instancias_teste/swiss42.txt', 'instancias/instancias_teste/bayg29.txt', 'instancias/instancias_teste/bier127.txt', 'instancias/instancias_teste/descricao.txt']


### ch150

In [156]:
graph = Graph(150)
graph.build(DATA_TEST[0], _type='tsp_test')
print(graph.num_vertices())

150


#### Nearest Neighbor

In [157]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = nearest_neighbor(graph, random_index)
print(cost, random_index)

8031 34


#### Variable Neighborhood Descent

In [158]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = vnd(graph, random_index, 2)
print(solution)
print(cost, random_index)

[46, 119, 41, 8, 27, 5, 36, 1, 18, 98, 113, 101, 107, 69, 134, 49, 57, 54, 64, 131, 136, 85, 28, 80, 109, 19, 24, 140, 82, 55, 145, 25, 74, 17, 84, 141, 142, 99, 4, 106, 94, 81, 102, 97, 0, 86, 75, 72, 47, 62, 29, 83, 6, 7, 88, 95, 34, 92, 51, 32, 104, 110, 15, 58, 78, 120, 87, 93, 9, 112, 2, 61, 148, 124, 21, 103, 3, 44, 70, 43, 114, 149, 20, 77, 14, 132, 76, 121, 13, 79, 71, 48, 146, 143, 128, 26, 30, 144, 135, 111, 63, 105, 12, 73, 122, 116, 56, 38, 40, 100, 115, 11, 23, 117, 52, 39, 138, 50, 108, 42, 66, 37, 22, 31, 130, 133, 137, 45, 89, 53, 91, 125, 123, 96, 33, 127, 67, 118, 90, 139, 59, 65, 16, 129, 147, 10, 60, 35, 68, 126, 46]
7946 40


### bays29

In [169]:
graph = Graph(29)
graph.build(DATA_TEST[1], _type='tsp_test')
print(graph.num_vertices())

29


#### Nearest Neighbor

In [170]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = nearest_neighbor(graph, random_index)
print(solution)
print(cost, random_index)

[20, 1, 19, 9, 3, 14, 17, 13, 21, 16, 10, 18, 15, 26, 23, 7, 0, 27, 5, 11, 8, 4, 25, 28, 2, 12, 24, 6, 22, 20]
2374 20


#### Variable Neighborhood Descent

In [173]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = vnd(graph, random_index, 2)
print(solution)
print(cost, random_index)

[8, 4, 25, 28, 2, 1, 20, 0, 5, 11, 27, 7, 26, 23, 15, 18, 14, 3, 9, 19, 12, 22, 6, 24, 10, 21, 13, 17, 16, 8]
2435 10


### berlin52

In [ ]:
graph = Graph(52)
graph.build(DATA_TEST[2], _type='tsp_test')
print(graph.num_vertices())

#### Nearest Neighbor

In [ ]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = nearest_neighbor(graph, random_index)
print(cost, random_index)

#### Variable Neighborhood Descent

In [ ]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = vnd(graph, random_index, 5)
print(cost, random_index)

### ch130

In [ ]:
graph = Graph(130)
graph.build(DATA_TEST[3], _type='tsp_test')
print(graph.num_vertices())

#### Nearest Neighbor

In [ ]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = nearest_neighbor(graph, random_index)
print(cost, random_index)

#### Variable Neighborhood Descent

In [ ]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = vnd(graph, random_index, 5)
print(cost, random_index)

### brazil58

In [ ]:
graph = Graph(58)
graph.build(DATA_TEST[4], _type='tsp_test')
print(graph.num_vertices())

#### Nearest Neighbor

In [ ]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = nearest_neighbor(graph, random_index)
print(cost, random_index)

#### Variable Neighborhood Descent

In [ ]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = vnd(graph, random_index, 5)
print(cost, random_index)

### swiss42

In [ ]:
graph = Graph(42)
graph.build(DATA_TEST[5], _type='tsp_test')
print(graph.num_vertices())

#### Nearest Neighbor

In [ ]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = nearest_neighbor(graph, random_index)
print(cost, random_index)

#### Variable Neighborhood Descent

In [ ]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = vnd(graph, random_index, 5)
print(cost, random_index)

### bayg29

In [ ]:
graph = Graph(29)
graph.build(DATA_TEST[6], _type='tsp_test')
print(graph.num_vertices())

#### Nearest Neighbor

In [ ]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = nearest_neighbor(graph, random_index)
print(cost, random_index)

#### Variable Neighborhood Descent

In [ ]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = vnd(graph, random_index, 5)
print(cost, random_index)

### bier127

In [ ]:
graph = Graph(127)
graph.build(DATA_TEST[7], _type='tsp_test')
print(graph.num_vertices())

#### Nearest Neighbor

In [ ]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = nearest_neighbor(graph, random_index)
print(cost, random_index)

#### Variable Neighborhood Descent

In [ ]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = vnd(graph, random_index, 5)
print(cost, random_index)

### TSP CUP

In [ ]:
graph = Graph(1500)
graph.build(DATA_CUP[0], _type='tsp_cup')
print(graph._adjacency_matrix)
print(graph.num_vertices())

In [ ]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = nearest_neighbor(graph, random_index)
print(cost, random_index)

In [ ]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = random_restart(graph, random_index, 100)
print(cost, random_index)

In [ ]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = best_improvement(graph, random_index)
print(cost, random_index)

In [ ]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = first_improvement(graph, random_index)
print(cost, random_index)

In [ ]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = vnd(graph, random_index, 10)
print(cost, random_index)